In [1]:
#Import libraries
import pyam
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [159]:
def get_data():
    #get IIASA identification
    #workplace IIASA
    iiasa_creds = r"C:\Users\scheifinger\Documents\GitHub\Feasibility_Tool\iiasa_credentials.yml"
    #workplace home
    #iiasa_creds = r"C:\Users\schei\OneDrive\Dokumente\GitHub\Feasibility_Tool\iiasa_credentials.yml"
    pyam.iiasa.Connection(creds=iiasa_creds)

    connections = list(pyam.iiasa.Connection(creds=iiasa_creds).valid_connections)
    #other variables: 'Emissions|CO2', 'Primary Energy|Coal', 
    #query for climate scenario data
    
    data_set = pyam.read_iiasa(
        name = 'engage_internal',
        creds = iiasa_creds,
        scenario =[
            "T34_1000_ref",
            "T34_1000_govem",
            "T34_1000_feas_em",
            "T34_1000_bitb_em",
            "T34_1000_bitb_ref",
            "T34_1000_enab_em"
        ],
        variable=["Emissions|CO2", 
                  'Emissions|Kyoto Gases',
                  'Primary Energy|Coal',
                  'Primary Energy|Gas',
                  'Primary Energy',
                  'Capacity|Electricity|Coal',
                  'Capacity|Electricity|Solar',
                  'Capacity|Electricity|Wind',
                  'Carbon Sequestration|CCS',
                  'Carbon Sequestration|Land Use'
                  ],
        region=["World",
                "North America; primarily the United States of America and Canada",
                "Eastern and Western Europe (i.e., the EU28)",
                "Countries of centrally-planned Asia; primarily China",
                "Countries of South Asia; primarily India", 
                "Countries of Sub-Saharan Africa",
                "Countries of Latin America and the Caribbean",
                "Countries of the Middle East; Iran, Iraq, Israel, Saudi Arabia, Qatar, etc."]
    )   
    #return data format of df
    return df


In [160]:
data_set = get_data().data

httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications/IXSE_ENGAGE_INTERNAL/config "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected to the IXSE_ENGAGE_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.

In [223]:
print("Sum of all countries is larger than World")
#TODO: ask/ check
print(
data_set[(data_set["model"] == "AIM/CGE V2.2") \
    & (data_set['scenario'] == "T34_1000_bitb_em") \
    & (data_set["variable"] == "Primary Energy|Gas") \
    & (data_set["year"] == 2020) \
    & (df["region"] != "World")
    ].agg({'value':"sum"})
)

data_set[(data_set["model"] == "AIM/CGE V2.2") \
    & (data_set['scenario'] == "T34_1000_bitb_em") \
    & (data_set["variable"] == "Primary Energy|Gas") \
    & (data_set["year"] == 2020)
    ]

Sum of all countries is larger than World
value    186.5678
dtype: float64


,model,scenario,region,variable,unit,year,value
154,AIM/CGE V2.2,T34_1000_bitb_em,Countries of Latin America and the Caribbean,Primary Energy|Gas,EJ/yr,2020,8.0877
325,AIM/CGE V2.2,T34_1000_bitb_em,Countries of South Asia; primarily India,Primary Energy|Gas,EJ/yr,2020,2.4281
496,AIM/CGE V2.2,T34_1000_bitb_em,Countries of Sub-Saharan Africa,Primary Energy|Gas,EJ/yr,2020,5.7682
667,AIM/CGE V2.2,T34_1000_bitb_em,Countries of centrally-planned Asia; primarily...,Primary Energy|Gas,EJ/yr,2020,9.5000
838,AIM/CGE V2.2,T34_1000_bitb_em,"Countries of the Middle East; Iran, Iraq, Isra...",Primary Energy|Gas,EJ/yr,2020,20.6728
1009,AIM/CGE V2.2,T34_1000_bitb_em,"Eastern and Western Europe (i.e., the EU28)",Primary Energy|Gas,EJ/yr,2020,17.7990
1180,AIM/CGE V2.2,T34_1000_bitb_em,North America; primarily the United States of ...,Primary Energy|Gas,EJ/yr,2020,32.9493
1351,AIM/CGE V2.2,T34_1000_bitb_em,World,Primary Energy|Gas,EJ/yr,2020,140.1110


In [218]:
#Data wrangling
df = data_set
#get regional groupings
## OECD: "North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)"
#TODO Clarify that countries of Asia and Latin America are missing
## China:  Countries of centrally-planned Asia; primarily China
## Rest of the world: other countries
#get world
world = data_set[df['region'] == "World"]
world.loc[:, "region"] = "World"
#get OECD*
oecd = df[df["region"].isin(["North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)"])]\
    .groupby(["model", "scenario", "variable", "year", "unit"])\
        .agg({"value": "sum"}).reset_index()
oecd['region'] = "OECD"
#get China
china = df[df['region'] == "Countries of centrally-planned Asia; primarily China"]
china.loc[:, "region"] = "China"
#get RoW
row = df[~df["region"].isin(["World", "North America; primarily the United States of America and Canada","Eastern and Western Europe (i.e., the EU28)", "Countries of centrally-planned Asia; primarily China"])]\
        .groupby(["model", "scenario", "variable", "year", "unit"])\
            .agg({"value": "sum"}).reset_index()
row["region"] = "RoW"
#Concat three regions
df = pd.concat([world, oecd, china, row]).reset_index()

#from long to wide
df = pd.pivot(data=df, index=['model','scenario', 'region', 'year'], columns = 'variable', values = 'value').reset_index()
#get diffeernt scenario narratives from scenario name
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_ref", case=True), "Cost Effective", np.nan) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_bitb_ref", case=True), "Tech", df['scenario_narrative']) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_govem", case=True), "Instit", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_bitb_em", case=True), "Tech+Inst", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_enab_em", case=True), "Inst+Enab", df['scenario_narrative'])
df['scenario_narrative'] = np.where(df['scenario'].str.contains("T34_1000_feas_em", case=True), "Tech+Inst+Enab", df['scenario_narrative'])

#caluclate percentages of energymix for coal and gas
df["Share_Coal"] = df["Primary Energy|Coal"] / df["Primary Energy"]
df["Share_Gas"] = df["Primary Energy|Gas"] / df["Primary Energy"]

#get reductions from 2020 - 2040
df.head(3)
#Sort dataframe
#df.sort_values(by=["model", "scenario", "region", "year"], ascending=True, inplace=True)

variable,model,scenario,region,year,Capacity|Electricity|Coal,Capacity|Electricity|Solar,Capacity|Electricity|Wind,Carbon Sequestration|CCS,Carbon Sequestration|Land Use,Emissions|CO2,Emissions|Kyoto Gases,Primary Energy,Primary Energy|Coal,Primary Energy|Gas,scenario_narrative,Share_Coal,Share_Gas
0,AIM/CGE V2.2,T34_1000_bitb_em,China,2010,773.8319,1.4445,14.6534,0.0,NaN,8860.3168,11537.5187,108.3570,76.8021,4.0615,Tech+Inst,0.708788,0.037483
1,AIM/CGE V2.2,T34_1000_bitb_em,China,2015,1052.4402,66.0579,61.9462,0.0,NaN,10778.5587,13745.7169,132.7931,92.2560,5.6674,Tech+Inst,0.694735,0.042678
2,AIM/CGE V2.2,T34_1000_bitb_em,China,2020,1418.3453,343.1131,153.9295,0.0,NaN,11958.4702,15070.7563,151.2920,97.7011,9.5000,Tech+Inst,0.645778,0.062792


In [153]:
#filter
to_plot_df = df[(df['year'].isin([2020, 2030, 2040])) & (df["scenario"].isin(["T34_1000_ref", "T34_1000_govem"])) & (df["region"] == "World")]

#calculate reductions
reductions_df = to_plot_df[["model", "scenario", "region", 'year', "Emissions|CO2"]]
reductions_df = pd.pivot(data=reductions_df, index=reductions_df.drop(['year', "Emissions|CO2"], axis=1).columns, columns = 'year', values = 'Emissions|CO2').reset_index()
reductions_df["2030_CO2_redu"] = ( (reductions_df[2030] - reductions_df[2020]) / reductions_df[2020]) * -1 #multiplied with -1 to have positive values 
reductions_df["2040_CO2_redu"] = ((reductions_df[2040] - reductions_df[2020]) / reductions_df[2020]) * -1 
reductions_df.drop([2020, 2030, 2040], axis=1, inplace=True)

#merge
to_plot_df = pd.merge(left=to_plot_df, right=reductions_df, on=["model", "scenario", "region"])

to_plot_df.head(3)

,model,scenario,region,year,Capacity|Electricity|Solar,Capacity|Electricity|Wind,Carbon Sequestration|CCS,Carbon Sequestration|Land Use,Emissions|CO2,Emissions|Kyoto Gases,Primary Energy,Primary Energy|Coal,Primary Energy|Gas,scenario_narrative,Share_Coal,Share_Gas,2030_CO2_redu,2040_CO2_redu
0,AIM/CGE V2.2,T34_1000_govem,World,2020,874.0862,578.1408,0.0000,NaN,41483.9891,57383.4183,605.7297,172.4136,140.0969,Instit,0.284638,0.231286,0.218336,0.467688
1,AIM/CGE V2.2,T34_1000_govem,World,2030,6308.6826,2317.6574,877.2622,NaN,32426.5506,43000.1260,589.2797,111.7377,151.4121,Instit,0.189617,0.256944,0.218336,0.467688
2,AIM/CGE V2.2,T34_1000_govem,World,2040,13187.4473,5052.0789,4738.5071,NaN,22082.4170,30603.7339,603.0775,59.8423,151.2554,Instit,0.099228,0.250806,0.218336,0.467688


In [158]:
fig = go.Figure(px.strip(
    to_plot_df[(to_plot_df['year'] == 2030)],
    x='scenario_narrative',
    y='2030_CO2_redu',
    color='model',
    stripmode='overlay'))


fig.add_trace(go.Box(
    y = to_plot_df[(to_plot_df['year'] == 2030) & (to_plot_df["scenario_narrative"] == "Instit")]["2030_CO2_redu"],
    name = "Instit",
    marker_color='grey',
    opacity=0.5,
    boxpoints=False,
    showlegend=False
))

fig.add_trace(go.Box(
    y = to_plot_df[(to_plot_df['year'] == 2030) & (to_plot_df["scenario_narrative"] == "Cost Effective")]["2030_CO2_redu"],
    name = "Cost Effective",
    marker_color='grey',
    opacity=0.3,
    boxpoints=False,
    showlegend=False
))

fig.update_traces({'marker':{'size': 8}})

fig.update_layout(autosize=False,
                  width=600,
                  height=600,
                  title = go.layout.Title(
                    text="ENGAGE 2C scenarios <br><sup>Global CO2 reductions by 2030</sup>",
                    xref="paper",
                    x=0),
                  xaxis_title="Scenario Narrative",
                  yaxis_title = "CO2 reductions",
                  yaxis_tickformat='.0%')
    
fig

In [244]:
#Feasibility constraints

#Coal reduction
#get media coal consumption in 2020
df[(df["year"] == 2030)].groupby(["region"])["Primary Energy|Coal"].describe()

,count,mean,std,min,25%,50%,75%,max
region,,,,,,,,
China,38.0,56.041418,14.467059,29.806632,48.286704,55.565989,64.209920,90.436352
OECD,38.0,6.890333,5.469974,1.100528,4.269227,6.438132,7.418936,27.848388
RoW,38.0,19.756672,5.740694,7.199248,16.353840,19.543754,22.371599,34.500193
World,38.0,98.085310,20.969411,49.714967,90.012625,98.770211,108.848852,145.496906


In [41]:
#Figure 1: Global CO2 trajectories
fig1 = px.line(df.sort_values(by=["model", "year"], ascending=False),
               x = "year",
               y="Emissions|CO2",
               color = "region",
               #line_dash="model",
               facet_col="scenario_narrative",
               facet_col_wrap= 3,
               range_x=[2005, 2100],
               range_y = [-10000, 15000],
               hover_data = {'scenario':True, 'model':True},
               title = "CO2 emission trajectories")


#fig1.update_layout(showlegend=False)
fig1

In [53]:
fig = go.Figure()

fig = go.Figure()
fig.add_trace(go.Box(
    x=to_plot_df["scenario_narrative"],
    y=to_plot_df["Emissions|CO2"],
    name="All Points",
    jitter=0.3,
    pointpos=0,
    boxpoints='all',  # represent all points
    #marker_color=to_plot_df["scenario_narrative"],
    line_color='rgb(7,40,89)'


))


fig2 = px.box(
    data_frame = to_plot_df
    ,y = 'Emissions|CO2'
    ,x = 'scenario_narrative'
    ,color = "scenario_narrative"
    ,points = 'all'
    ,hover_data = {'model':True, 'scenario_narrative':False}
)
fig2
 

In [122]:
#Figure 2: GHG trajectories
fig2 = px.line(df,
               x = "year",
               y="Emissions|Kyoto Gases",
               color = "region",
               line_dash="model",
               facet_col="scenario_narrative",
               facet_col_wrap= 3,
               range_x=[2005, 2100],
               range_y = [-10000, 20000],
               hover_data = {'scenario':True, 'model':True}, 
               title = "GHG emission trajectories")
fig2.update_layout(showlegend=False)
fig2

In [31]:
#calculate year that each scenario hit's net zero
netzero_df = df[df["Emissions|CO2"] <= 0].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
netzero_df.rename(columns={"year": "year_netzero"}, inplace=True)
netzero_df
#aggregated cost until 2100
#df["year"] = pd.to_numeric(df['year'])
#df[df["year"] <= 2100].groupby(["model", "scenario"]).agg({"AR5 climate diagnostics|Temperature|Exceedance Probability|1.5 degC|MAGICC6": ['sum']})

,model,scenario,carbon_budget,year_netzero
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2060
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2070
3,GEM-E3_V2023,T34_1000_feas_em,2C,2090
4,IMAGE 3.2,T34_1000_feas_em,2C,2070
5,IMAGE 3.2,T34_1000_feas_pr,2C,2070
6,IMAGE 3.2,T34_1000_feas_ref,2C,2070
7,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,2080
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,2070
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_ref,2C,2100


In [10]:
fig2 = px.scatter(
    data_frame=netzero_df,
    x = "year",
    y="carbon_budget"
)
fig2

In [32]:
#get amounts of unabated coal
#both Coal data are in EJ/yr
unabcoal_df = df[["model", "scenario", "carbon_budget", "region","year", "Primary Energy|Coal", "Primary Energy|Coal|w/ CCS"]] 
unabcoal_df["Unabated Coal"] = unabcoal_df["Primary Energy|Coal"] - unabcoal_df["Primary Energy|Coal|w/ CCS"]
#pick threshold for when coal is supposed to the phased out // now it's 8EJ/y
unabcoal_df = unabcoal_df[unabcoal_df["Unabated Coal"] <= 8].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
unabcoal_df.rename(columns={"year": "year_netcoal"}, inplace=True)
#combine with netzero df

unabcoal_df

C:\Users\schei\AppData\Local\Temp\ipykernel_10492\1471908738.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,model,scenario,carbon_budget,year_netcoal
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2055
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2050
3,GEM-E3_V2023,T34_1000_feas_em,2C,2065
4,GEM-E3_V2023,T34_1000_feas_ref,2C,2080
5,IMAGE 3.2,T34_1000_feas_em,2C,2060
6,IMAGE 3.2,T34_1000_feas_pr,2C,2060
7,IMAGE 3.2,T34_1000_feas_ref,2C,2060
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,2060
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,2060


In [88]:
#combine dataframes
to_plot = pd.merge(left=netzero_df, right=unabcoal_df, on=["model", "scenario", "carbon_budget"])

fig3 = px.scatter(
    data_frame=to_plot,
    x = "year_netzero",
    y="year_netcoal",
    color= "carbon_budget"
)
fig3

In [89]:
#calculate change in transportation energy from 2020 to 2030
tranred_df = df[["model", "scenario", "carbon_budget", "region","year", "Final Energy|Transportation"]]
#filter for year 2020 & 2030
tranred_df = tranred_df[(tranred_df["year"] == 2020) | (tranred_df["year"] == 2030) ].reset_index().drop("index", axis=1)
#from long to wide
tranred_df = pd.pivot(data=tranred_df, index=["model", "scenario", "carbon_budget", "region"], 
             columns="year", values= "Final Energy|Transportation").reset_index()
#Finale Energy Transportation = (FET)
tranred_df["redu_FET"] = (tranred_df[2030] - tranred_df[2020]) / tranred_df[2020] 

# #both Coal data are in EJ/yr
# unabcoal_df = df[["model", "scenario", "carbon_budget", "region","year", "Primary Energy|Coal", "Primary Energy|Coal|w/ CCS"]] 
# unabcoal_df["Unabated Coal"] = unabcoal_df["Primary Energy|Coal"] - unabcoal_df["Primary Energy|Coal|w/ CCS"]
# #pick threshold for when coal is supposed to the phased out // now it's 8EJ/y
# unabcoal_df = unabcoal_df[unabcoal_df["Unabated Coal"] <= 8].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
# unabcoal_df.rename(columns={"year": "year_netcoal"}, inplace=True)

px.line(df,
        x = "year",
        y="Final Energy|Transportation",
        color = "carbon_budget",
        line_dash="model",
        facet_col="scenario_narrative",
        hover_data = {'scenario':True, 'model':True})

tranred_df

year,model,scenario,carbon_budget,region,2020,2030,redu_FET
0,AIM/CGE V2.2,T34_1000_feas_em,2C,World,123.601600,104.300200,-0.156158
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,World,123.601600,104.296500,-0.156188
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,World,123.601600,111.827600,-0.095258
3,GEM-E3_V2023,T34_1000_feas_em,2C,World,106.172003,107.829240,0.015609
4,GEM-E3_V2023,T34_1000_feas_ref,2C,World,106.172003,107.223507,0.009904
5,IMAGE 3.2,T34_1000_feas_em,2C,World,92.399406,78.249680,-0.153137
6,IMAGE 3.2,T34_1000_feas_pr,2C,World,92.399406,77.118727,-0.165376
7,IMAGE 3.2,T34_1000_feas_ref,2C,World,92.399406,85.292977,-0.076910
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,World,113.118992,140.618740,0.243105
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,World,113.118992,141.497312,0.250871


In [91]:
#merge reduction data to rest of df
to_plot = pd.merge(left=netzero_df, right=unabcoal_df, on=["model", "scenario", "carbon_budget"])
to_plot = pd.merge(left=to_plot, right=tranred_df.loc[:, ~tranred_df.columns.isin([2020, 2030])],
                   on=["model", "scenario", "carbon_budget"])
to_plot.head(3)

,model,scenario,carbon_budget,year_netzero,year_netcoal,region,redu_FET
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2060,2055,World,-0.156158
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055,2055,World,-0.156188
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2070,2050,World,-0.095258


In [86]:
fig4 = px.scatter(
    data_frame=to_plot,
    y = "redu_FET",
    x="year_netcoal",
    color= "carbon_budget"
)
fig4

In [102]:
#Statement
#Until when is it feasible to phase out coal
year_coal_phaseout = 2050
#What change in energy consumption of transport sector is realistic until 2030?
change_energy_transport = 0

#filter dataframe
filter_df = to_plot.loc[(to_plot["year_netcoal"] >= year_coal_phaseout) & (to_plot["redu_FET"] >= change_energy_transport)]
earliest_net_zero_year = filter_df["year_netzero"].min()
PA_aligned = (filter_data_set["carbon_budget"].str.contains("1.5C").sum() > 0)

print("OUTPUT:")
print("Earliest possible net-zero year:", earliest_net_zero_year)
print("Is PA still possible?", PA_aligned)

OUTPUT:
Earliest possible net-zero year: 2050
Is PA still possible? True
